In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!pip install -q -U transformers peft accelerate optimum
!pip install datasets==2.15.0
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu117/ # gptq
# !pip install -q -U https://github.com/casper-hansen/AutoAWQ/releases/download/v0.1.6/autoawq-0.1.6+cu118-cp310-cp310-linux_x86_64.whl # awq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig
import psutil
import time
import torch, math
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
from datasets import load_dataset
from torch.utils.data import DataLoader

# Quantize - GPTQ

In [ ]:
import huggingface_hub
huggingface_hub.login()

In [ ]:
ds = load_dataset("roneneldan/TinyStories")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
class Collator():
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, batchs):
        tokens = self.tokenizer(text=batchs, return_tensors="pt", padding=True)
        return tokens

In [ ]:
def calculate_perplexity(model, tokenizer, dataloader):
    perplexity = 0
    for batch in dataloader:
        batch = {k: v.to(0) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch, labels=batch['input_ids'])
            loss = outputs.loss.item()
            perplexity += math.exp(loss)
    return perplexity / len(dataloader)

In [ ]:
model_path = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

gptq_config = GPTQConfig(bits=4, group_size=128, dataset=ds['train']['text'][:100], desc_act=False, tokenizer=tokenizer,)

gptq_model = AutoModelForCausalLM.from_pretrained(model_path, quantization_config=gptq_config, device_map='auto')

Quantizing model.layers blocks :   0%|          | 0/16 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
collator = Collator(tokenizer)
train_dataset, validation_dataset = load_dataset("roneneldan/TinyStories", split=['train', 'validation'])
train_dataloader = DataLoader(train_dataset['text'][:100], batch_size=8, collate_fn=collator)

base_avg_perplexity = calculate_perplexity(gptq_model, tokenizer, train_dataloader)
print(base_avg_perplexity)
gptq_model.save_pretrained('/content/gdrive/MyDrive/dinolabs/dinolabs3rd/llama3.2-1B-base')

14.884504415288122


In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_path).to(0)
original_perplexity = calculate_perplexity(model, tokenizer, train_dataloader)

In [ ]:
print(original_perplexity)

44.59020990003144


In [ ]:
# awq_config = {"zero_point": True, "q_group_size": 128, "w_bit": 4, "version":"GEMM"}

# # Load model
# awq_model = AutoAWQForCausalLM.from_pretrained(model_path)

# # Quantize
# awq_model.quantize(tokenizer, quant_config=awq_config)